## Agile Hardware Design
***
# Functional Programming Primer

## Prof. Scott Beamer
### sbeamer@ucsc.edu

## [CSE 293](https://classes.soe.ucsc.edu/cse293/Winter22/)

## Plan for Today

* _Big Idea:_ applying functions to collections of elements
* Anonymous functions in Scala
* Scala `map`, `foreach`, `zip` operators
* Chisel example

## Loading The Chisel Library Into a Notebook

In [ ]:
val path = System.getProperty("user.dir") + "/../resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

## Why Use Functional Programming with Chisel?

* Chisel's power comes from its ability to make parameterized hardare generators

* Functional programming (FP) operations ease dealing with _functions over collections_
  * Using standard features/patterns improves productivity, readability, and correctness

* Being forced to break problem into standard patterns may help with reasoning
  * Additionally, compiler may be able to spot more errors

* Be mindful of side-effects and how to clearly convey them in code
  * FP operations typically intended for side-effect free programming
  * Chisel operations often have deliberate side effects (e.g. connecting things)

## Motivation for Working Over Collections

* Arguably, much of programming (and hardware design) works over collections rather than scalar values
  * Collections aggregate similar things

* Often, we want to apply an operation to everything in the collection
  * Traditionally, we use iteration (e.g. `for` loops)

* _Problem:_ every usage has to reinvent wheel

* What if ...
  * We could recognize _patterns_ of function application and use those?
  * We could use the compiler to check compliance with those patterns?

## Solution - Reuse through Patterns

* Note these operations are not core language constructs, but instead methods defined for these collections
* Only some operations covered today (more to come)

<img src="images/map+foreach.svg" alt="map & foreach viz" style="width:80%;margin-left:auto;margin-right:auto"/>

## Anonymous Functions in Scala

* Technically called _function literals_

* Can bind to a name, but often will use within other construct and never name it explicitly (_anonymous_)

* Syntax - argument list in parentheses on left, `=>`, function body on right
```scala
    (x: Int) => x + 1
```

In [ ]:
(x: Int) => x + 1
val inc = (x: Int) => x + 1
def inc2(x: Int) = x+1
inc(2)
inc2(2)
(a: Int, b: Int) => a+b

## `map` in Scala

```scala
l map f
```

* Applies given function to each element and returns result as new collection
* Should not make assumptions about order in which it is applied

In [ ]:
def inc(x: Int) = x+1

val l = 0 until 5
l.map(inc)
l map inc
l map { i => inc(i) }
l map { i => i + 1 }

## `foreach` in Scala

```scala
l foreach f
```

* Applies given function to each element (like map), but does not return anything (unlike map)
* Useful for indicating intent is the side effect and not the result

In [ ]:
val l = 0 until 5

l foreach println

## Using `map` and `foreach` in Chisel

* Operations available on both Scala collections (e.g. `Seq`) and Chisel aggregates (e.g. `Vec`)

In [ ]:
class ConstOut(numElems: Int, const: Int) extends Module {
    val io = IO(new Bundle {
        val out = Output(Vec(numElems, UInt()))
    })

    val seqOfInts = 0 until numElems
    val seqOfUInts = seqOfInts map { i => i.U }
    
    io.out foreach { o => o := const.U }
}

println(getVerilog(new ConstOut(2,8)))

## Scala Tuples

* Can group together heterogeneous things
  * Doesn't name members, but can index them numerically (**starts from 1**)
  * Often can pattern match (with `case`) or assign to access members

* Best when number of things is small and producer/consumers are nearby
  * Suggest case class to explicitly name members (for readability)
  * Suggest collection for many elements (for manageability)

In [ ]:
val t1 = (2,3)
val t2 = ("My", 8)
t1._1
t1._2
val (a,b) = t1

## `zip` in Scala

```scala
l1 zip l2
```

* Pairs up elements with elements of another collection
* Commonly used to join together collections before applying other operations
* _Note:_ If collections have different sizes, result is the minimum size

In [ ]:
val l1 = 0 until 5
l1 zip l1
l1 zip Seq(8)
l1 zip l1 map {case (a,b) => a+b}

## `zip` Diagram

<img src="images/zip.svg" alt="zip viz" style="width:70%;margin-left:auto;margin-right:auto"/>

## Chisel Example Using `foreach` and `zip`

In [ ]:
class VecAbs(numElems: Int, width: Int) extends Module {
    val io = IO(new Bundle {
        val in = Input(Vec(numElems, SInt(width.W)))
        val out = Output(Vec(numElems, SInt(width.W)))
    })

    def abs(x: SInt): SInt = Mux(x < 0.S, -x, x)

//     for (i <- 0 until numElems) {
//         io.out(i) := abs(io.in(i))
//     }

    io.out.zip(io.in) foreach { case (o,i) => o := abs(i) }
}

println(getVerilog(new VecAbs(2,8)))

## Scala (fuction) Placeholders

* Able to make function literals even more concise by not explicitly naming arguments and then using them
* Use `_` in place of argument, and each use advances to next argument
* Use **CAREFULLY** to shorten code to improve readability
  * If intent not immediately clear, fall back to explicitly naming arguments

In [ ]:
val l = 0 until 5
l map { i => i + 1 }
l map { _ + 1 }

## Redoing Our Arbiter with FP (1/2)

In [ ]:
class MyArb(numPorts: Int, n: Int) extends Module {
    val io = IO(new Bundle {
        val req = Flipped(Vec(numPorts, Decoupled(UInt(n.W))))
        val out = Decoupled(UInt(n.W))
    })
    require (numPorts > 0)
    val inValids = Wire(Vec(numPorts, Bool()))
    val inBits   = Wire(Vec(numPorts, UInt(n.W)))
    val chosenOH = PriorityEncoderOH(inValids)
    for (p <- 0 until numPorts) {
        io.req(p).ready := chosenOH(p) && io.out.fire
        inValids(p) := io.req(p).valid
        inBits(p) := io.req(p).bits
    }
    io.out.valid := inValids.asUInt.orR
    io.out.bits := Mux1H(chosenOH, inBits)
}

## Redoing Our Arbiter (2/2)

In [ ]:
class MyArb(numPorts: Int, n: Int) extends Module {
    val io = IO(new Bundle {
        val req = Flipped(Vec(numPorts, Decoupled(UInt(n.W))))
        val out = Decoupled(UInt(n.W))
    })
    require (numPorts > 0)
    val inValids = io.req map { _.valid }
    io.out.valid := VecInit(inValids).asUInt.orR
    val chosenOH = PriorityEncoderOH(inValids)
    io.out.bits := Mux1H(chosenOH, io.req map { _.bits })
    io.req.zip(chosenOH) foreach { case (i, c) => i.ready := c && io.out.fire}
}